# Optimization

of linear-quadratic problems, using the [OSQP.jl](https://github.com/osqp/OSQP.jl) package.

The example is (for pedagogical reasons) the same as in the other notebooks on optimization. Otherwise, the methods illustrated here are well suited for cases when the objective involves the portfolio variance ($ w'\Sigma w $) or when the estimation problem is based on minimizing the sum of squared residuals ($u'u$), and the restrictions are linear expressions.

## Load Packages and Utility Functions

In [1]:
using Printf, LinearAlgebra, SparseArrays, OSQP

include("jlFiles/printmat.jl");

# The OSQP.jl Optimization Package

The [OSQP.jl](https://github.com/osqp/OSQP.jl) package is tailor made for solving linear-quadratic problems (with linear restrictions). It solves problems of the type

$\min 0.5\theta' P \theta + q' \theta$ subject to $l \leq A \theta \leq u$, 

where $\theta$ is a vector of choice variables.

To get an equality restriction in row `i`, set `l[i]=u[i]`.

Notice that $(P,A)$ to should be `Sparse` matrices and $(q,l,u)$ vectors with `Float64` numbers.

# A Linear-Quadratic Minimization Problem

We specify a matrix $P$ and a vector $q$ needed by OSQP (see above).

We consider several cases below: no restrictions on $\theta$, bounds on $\theta$, and a linear equality restriction.

## Unconstrained Minimization

In [2]:
P = 2*[1  0;           #we want to minimize b'[]*b + q'b, hence the 2*[]
       0  16]          #this is the same as minimizing (x-2)^2 + (4y+3)^2
q =   [-4.0, 24.0]     #vector, Float64

A = zeros(1,2)         #effectively no restriction
l = [0.0]              #vectors, Float64
u = [0.0]

settings = Dict(:verbose => true)
model = OSQP.Model()
OSQP.setup!(model; P=sparse(P), q=q, A=sparse(A), l=l, u=u, settings...)
result = OSQP.solve!(model)

printblue("Unconstrained minimization: the solution should be (2,-3/4):\n")
printmat(result.x)

Unconstrained minimization: the solution should be (2,-3/4):

     2.000
    -0.750

-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 2, constraints m = 1
          nnz(P) + nnz(A) = 2
settings: linear system solver = qdldl,
          eps_abs = 1.0e-003, eps_rel = 1.0e-003,
          eps_prim_inf = 1.0e-004, eps_dual_inf = 1.0e-004,
          rho = 1.00e-001 (adaptive),
          sigma = 1.00e-006, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter  objective    pri res    dua res    rho        time
   1 -8.3200e+000  0.00e+000  1.44e+001  1.00e-001  2.70e-005s
  25 -1.3000e+001  0.00e

## Constrained Minimization

Bounds on the solution: $2.75 \le \theta_1$ and $\theta_2 \le -0.3$.

In [3]:
A = I[1:2,1:2]     #identity matrix, I₂
l = [2.75,-Inf]    #2.75 <= θ₁ <= Inf, -Inf <= θ₂ <= -0.3
u = [Inf,-0.3]

settings = Dict(:verbose => false)
model = OSQP.Model()
OSQP.setup!(model; P=sparse(P), q=q, A=sparse(A), l=l, u=u, settings...)
result = OSQP.solve!(model)

printblue("with bounds on the solution: the solution should be (2.75,-0.75):\n")
printmat(result.x)

with bounds on the solution: the solution should be (2.75,-0.75):

     2.750
    -0.750



## Constrained Minimization

A linear equality constraint: $\theta_1 + 2\theta_2 = 3$.

In [4]:
A = [1 2]                    #equality constraint
l = [3.0]
u = [3.0]

model = OSQP.Model()
OSQP.setup!(model; P=sparse(P), q=q, A=sparse(A), l=l, u=u, settings...)
result = OSQP.solve!(model)

printblue("equality constraint: the solution should be (4,-1/2):\n")
printmat(result.x)

equality constraint: the solution should be (4,-1/2):

     4.000
    -0.500



# Updating the Problem (extra)

When you just want to change some of the inputs to the optimization problem, then it may pay off to do `update!()`. This is especially useful when you resolve the problem many times (in a loop, say).

In [5]:
model = OSQP.Model()                #setting up the problem for the first time
OSQP.setup!(model; P=sparse(P), q=q, A=sparse(A), l=l, u=u, settings...)
result = OSQP.solve!(model)
printmat(result.x)

q_new = [-2.0,20.0]                 #resolve the problem with different q values
OSQP.update!(model;q=q_new)         #(partially) update the problem      
result = OSQP.solve!(model)
printmat(result.x)

     4.000
    -0.500

     3.600
    -0.300



# An Alternative Package

The [Clarabel](https://github.com/oxfordcontrol/Clarabel.jl) package is an interesting alternative. Is has a slightly different syntax, but the the `DoClarabel()` function in `jlFiles/DoClarabel.jl` provides a way to call it using the same syntax as for OSQP.jl.